In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import shap
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap
from explainerdashboard import ExplainerDashboard, ClassifierExplainer
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import captum.attr as c
import time
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import os
import joblib


#Tried XGBoost to increase the performance of the overall model
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

#Assessing Gflops and Runtime
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('../Cleaned_full_data.csv')

C:\Users\matyc\AppData\Local\Temp\ipykernel_13072\3121833706.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.


In [3]:
# Reset retained index.
data = data.reset_index(drop=True)
# Set NA to 0.
data['ct_ftp_cmd'] = data['ct_ftp_cmd'].fillna(0)
data['attack_cat'] = data['attack_cat'].str.replace(r'\s+', '', regex=True)
data['attack_cat'] = data['attack_cat'].str.replace('Backdoors', 'Backdoor')

In [4]:
data = data.drop(columns=['proto', 'dsport', 'service', 'state', 'srcip', 'sport', 'dstip'])

In [5]:
temp = data[['is_ftp_login', 'is_sm_ips_ports', 'label', 'attack_cat']]
data = data.drop(columns=['is_ftp_login', 'is_sm_ips_ports', 'label', 'attack_cat'])

In [6]:
ohe1 = pd.read_csv('../Full_proto_encoded.csv')
ohe2 = pd.read_csv('../Full_dsport_encoded.csv')
ohe3 = pd.read_csv('../Full_service_encoded.csv')
ohe4 = pd.read_csv('../Full_state_encoded.csv')
# Spelling error.
ohe5 = pd.read_csv('../Full_scrip_encoded.csv')
#------------------------------------------#
ohe6 = pd.read_csv('../Full_sport_encoded.csv')
ohe7 = pd.read_csv('../Full_dstip_encoded.csv')

In [7]:
# MinMax seperates Normal data well and reduces noise. Please see Kmeans TSNE evaluation in Archive.
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
data = pd.DataFrame(scaled_data, columns=data.columns)
data = pd.concat([data, temp, ohe1, ohe2, ohe3, ohe4, ohe5, ohe6, ohe7], axis=1)

In [8]:
# Assuming 'data' is your DataFrame and 'attack_cat' is the target variable
X = data.drop(columns=['label', 'attack_cat'])
y = data['attack_cat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Removing Label Encoder for CatBoost
#label_encoder = LabelEncoder()
#y_train = label_encoder.fit_transform(y_train)
#y_test = label_encoder.transform(y_test)
#path = './saved_models/label_encoder.joblib'
#joblib.dump(label_encoder, path)

print('Train:', pd.Series(y_train).value_counts())
print(len(pd.Series(y_train).value_counts()))
print('Test:', pd.Series(y_test).value_counts())
print(len(pd.Series(y_test).value_counts()))

Train: attack_cat
Normal            1774933
Generic            172371
Exploits            35793
Fuzzers             19462
DoS                 13038
Reconnaissance      11137
Analysis             2127
Backdoor             1852
Shellcode            1191
Worms                 133
Name: count, dtype: int64
10
Test: attack_cat
Normal            443831
Generic            43110
Exploits            8732
Fuzzers             4784
DoS                 3315
Reconnaissance      2850
Analysis             550
Backdoor             477
Shellcode            320
Worms                 41
Name: count, dtype: int64
10


In [9]:
# Random Forest
#rf = RandomForestClassifier(random_state=42)

# XGBoost 
#rf = XGBClassifier(eval_metric='mlogloss', random_state=42)

# ExtraTrees
#rf = ExtraTreesClassifier(n_estimators=200, random_state=42)

# Catboost
rf = CatBoostClassifier(random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print('\nClassification Report (Test Set):')
#print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print(classification_report(y_test, y_pred))

# Time the prediction
start = time.perf_counter()
end = time.perf_counter()

# Estimate FLOPs (approximation for tree-based models)
n_samples = X_test.shape[0]
n_trees = rf.tree_count_
avg_depth = 10  # use actual 'depth' if you know it

flops = n_samples * n_trees * avg_depth
gflops = flops / ((end - start) * 1e9)

print(f"Time taken: {end - start:.6f} seconds")
print(f"Estimated GFLOPS: {gflops:.6f}")



Learning rate set to 0.116688
0:	learn: 1.3775228	total: 1.9s	remaining: 31m 42s
1:	learn: 1.0760025	total: 3.38s	remaining: 28m 5s
2:	learn: 0.8818764	total: 4.87s	remaining: 26m 58s
3:	learn: 0.7411670	total: 6.44s	remaining: 26m 44s
4:	learn: 0.6341834	total: 8s	remaining: 26m 32s
5:	learn: 0.5496430	total: 9.5s	remaining: 26m 13s
6:	learn: 0.4798978	total: 11s	remaining: 25m 58s
7:	learn: 0.4226119	total: 12.5s	remaining: 25m 45s
8:	learn: 0.3747260	total: 14s	remaining: 25m 37s
9:	learn: 0.3339212	total: 15.4s	remaining: 25m 26s
10:	learn: 0.2993629	total: 17s	remaining: 25m 26s
11:	learn: 0.2696870	total: 18.5s	remaining: 25m 25s
12:	learn: 0.2441238	total: 20s	remaining: 25m 19s
13:	learn: 0.2220479	total: 21.5s	remaining: 25m 14s
14:	learn: 0.2028109	total: 24.6s	remaining: 26m 54s
15:	learn: 0.1855405	total: 26.1s	remaining: 26m 45s
16:	learn: 0.1709118	total: 28.9s	remaining: 27m 50s
17:	learn: 0.1578025	total: 30.3s	remaining: 27m 34s
18:	learn: 0.1465074	total: 31.8s	remain

ZeroDivisionError: float division by zero

In [11]:
runs = 100
start = time.perf_counter()
for _ in range(runs):
    rf.predict(X_test)
end = time.perf_counter()

elapsed = (end - start) / runs

flops = n_samples * n_trees * avg_depth
gflops = flops / (elapsed * 1e9)

print(f"Average Time per Run: {elapsed:.6f} seconds")
print(f"Estimated GFLOPS: {gflops:.6f}")

Average Time per Run: 0.562331 seconds
Estimated GFLOPS: 9.034005


In [12]:
#Label Encoded Version
#accuracy_per_label = {}
#for i, label in enumerate(label_encoder.classes_):
#    true_label_indices = np.where(y_test == i)[0]
#    y_pred_for_label = y_pred[true_label_indices]
#    correct = np.sum(y_pred_for_label == i)
#    total = len(true_label_indices)
#    accuracy_per_label[label] = correct / total
#print("\nAccuracy per label:")
#for label, acc in accuracy_per_label.items():
#    print(f"{label}: {acc:.4f}")


#Non Label Encoded Version
accuracy_per_label = {}
labels = np.unique(y_test)  # Get unique class labels directly

for label in labels:
    true_label_indices = np.where(y_test == label)[0]
    y_pred_for_label = y_pred[true_label_indices]
    correct = np.sum(y_pred_for_label == label)
    total = len(true_label_indices)
    accuracy_per_label[label] = correct / total

print("\nAccuracy per label:")
for label, acc in accuracy_per_label.items():
    print(f"{label}: {acc:.4f}")


Accuracy per label:
Analysis: 0.0200
Backdoor: 0.0608
DoS: 0.2998
Exploits: 0.8302
Fuzzers: 0.6298
Generic: 0.9846
Normal: 0.9979
Reconnaissance: 0.7898
Shellcode: 0.8562
Worms: 0.0732


In [ ]:
#model_filename = './saved_models/SecondaryMulti.joblib'
#joblib.dump(rf, model_filename)